In [1]:
#  Last amended: 15th August, 2019
#  Myfolder: /home/ashok/Documents/spark
# Ref:
# Tutorials:
#      https://changhsinlee.com/pyspark-dataframe-basics/
#      https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
#      https://dzone.com/articles/pyspark-dataframe-tutorial-introduction-to-datafra
# Documentation:
#      http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module
#      http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions
# Cheat Sheet
#      https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

#  Objectives:
#           Dataframe operations in spark cluster
#           Dealing with categorical columns
#               i) StrinIndexing them
#              ii) OneHotEncoding
#             iii) PipeLining for multiple columns

A. Initial operations:
1.0 Start hadoop in a terminal:

            ./allstart.sh

In [ ]:
# 1.1 Transfer two data files, airports.csv and  weather.csv,
#         from local file system to hadoop. 
#         COPY AND PASTE in terminal all the four commands below.
"""

cd ~
hdfs dfs -put /cdata/nycflights13/airports.csv hdfs://localhost:9000/user/ashok/data_files/nycflights
hdfs dfs -put /cdata/nycflights13/weather.csv hdfs://localhost:9000/user/ashok/data_files/nycflights
hdfs dfs -ls -h hdfs://localhost:9000/user/ashok/data_files/nycflights

"""

In [ ]:
# 1.2 Start pyspark in terminal. Its starting creates a context 'sc' and session 'spark'
#     Jupyter notebook will also start then. pyspark is using juptyter notebook as interface
#     COPY AND PASTE FOLLOWING TWO COMMANDS
"""

cd /home/ashok/Documents/spark/1.basics/
pyspark

"""

In [1]:
# Display multiple outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Increase cell width to display wide columnar output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# 1.3 To check if spark context is ,sc, and spark session is, spark: 
#        try the following two commands. Also right-click on Spark_UI hyperlink
#        to open another site.
#      Spark 'driver' process manifests itself through 'spark' session object

sc
spark

<SparkContext master=local[*] appName=PySparkShell>

### Spark DataFrame creation

In [4]:
# 1.4 Using 'spark' session, create a range of numbers 

myRange = spark.range(10000)
type(myRange)
myRange.show(3)

pyspark.sql.dataframe.DataFrame

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+
only showing top 3 rows



In [5]:
#  Use createDataFrame() . This is preferred method.
#  1.4.1 From a list of tuples:
#  1.4.1 No schema. Column names are given on its own
df = spark.createDataFrame([(11,12,31), (4,5,6)])
df.show()
# 1.4.2 Specify schema
df = spark.createDataFrame([(1,2,3), (4,5,6)], schema = ['age', 'income','ht']).show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
| 11| 12| 31|
|  4|  5|  6|
+---+---+---+

+---+------+---+
|age|income| ht|
+---+------+---+
|  1|     2|  3|
|  4|     5|  6|
+---+------+---+



In [6]:
# 1.5.1 From a list of dictionaries:

df = spark.createDataFrame([ {'name':['Alice', 'kishan'], 'age' : [22,44]}]).show()

df = spark.createDataFrame([ {'name':['Alice'], 'age' : [22]},
                             {'name':['kishan'], 'age' : [44]}]).show()

df = spark.createDataFrame([ {'name':'Alice', 'age' : 22},
                             {'name':'kishan' , 'age' : 44}]).show()


/opt/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+---------------+
|     age|           name|
+--------+---------------+
|[22, 44]|[Alice, kishan]|
+--------+---------------+

+----+--------+
| age|    name|
+----+--------+
|[22]| [Alice]|
|[44]|[kishan]|
+----+--------+

+---+------+
|age|  name|
+---+------+
| 22| Alice|
| 44|kishan|
+---+------+



In [8]:
# 1.5.3 Using Pandas

import pandas as pd
import numpy as np
pdf = pd.DataFrame({'age' : np.random.randint(20,50, 10000),    
                    'married' : np.random.randint(0,2,10000)
                    }
                  )
pdf.head(3)

#  1.5.4 Transform it to spark dataframe
#        Note that unlike pandas, there is no concept
#        of index here.
big = spark.createDataFrame(pdf)
big.show(3)
big.count()   # How many rows

,age,married
0,29,0
1,43,0
2,34,0


+---+-------+
|age|married|
+---+-------+
| 29|      0|
| 43|      0|
| 34|      0|
+---+-------+
only showing top 3 rows



10000

In [9]:
# 1.5.5 Using 'Row' object of dataset
from pyspark.sql import Row

# Create an RDD
df = sc.parallelize([ 
                     Row(name='Alice', age=5, height=80), 
                     Row(name='Alice', age=5, height=80), 
                     Row(name='Alice', age=10, height=80),
                     Row(name=None, age=None, height=None)
                     ]
                    )

type(df)      #
df

pyspark.rdd.RDD

ParallelCollectionRDD[80] at parallelize at PythonRDD.scala:195

In [10]:
# 1.5.6 Transform to dataframe and process
df1 = df.toDF()
type(df1)
df1.dropDuplicates().show()
df1.dropna().show()
df1.dtypes

pyspark.sql.dataframe.DataFrame

+----+------+-----+
| age|height| name|
+----+------+-----+
|   5|    80|Alice|
|null|  null| null|
|  10|    80|Alice|
+----+------+-----+

+---+------+-----+
|age|height| name|
+---+------+-----+
|  5|    80|Alice|
|  5|    80|Alice|
| 10|    80|Alice|
+---+------+-----+



[('age', 'bigint'), ('height', 'bigint'), ('name', 'string')]

In [12]:
# 1.5.7  Take a stratified sample from DataFrame:
#              80% of 0s and 10% of 1s
#              We have to decide how stratification will be done.

sample = big.sampleBy("married", fractions = {0 : 0.8, 1 : 0.2 })
type(sample)
# 1.5.8
big.count()
sample.count()
# 1.5.9
sample.groupby('married').count().show()   # Same as value_counts() in pandas
big.groupby('married').count().show()

pyspark.sql.dataframe.DataFrame

10000

4946

+-------+-----+
|married|count|
+-------+-----+
|      0| 3924|
|      1| 1022|
+-------+-----+

+-------+-----+
|married|count|
+-------+-----+
|      0| 4907|
|      1| 5093|
+-------+-----+



In [15]:
###### B. Read Dataset from hadoop
# 2.0 Read 'airports.csv file into spark from hadoop

# Where is my file? Path:
URL_of_file= "hdfs://localhost:9000/user/ashok/data_files/nycflights/"

# 2.0.1 Takes time. We use 'spark' session object to read file:
airports_df = spark.read.csv(path = URL_of_file + "airports.csv",
                             inferSchema = True,
                             header = True
                            )

In [16]:
# 2.1 Show top-5 rows
# 2.1.1
airports_df.head(3)         #    List of four 'Row' instances. 
airports_df.take(3)         #    Result same as above but prefer take() to head()
airports_df.collect()       #   collect() takes no number and returns all rows


[Row(faa='04G', name='Lansdowne Airport', lat=41.1304722, lon=-80.6195833, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605722, lon=-85.6800278, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.9893408, lon=-88.1012428, alt=801, tz=-6, dst='A')]

[Row(faa='04G', name='Lansdowne Airport', lat=41.1304722, lon=-80.6195833, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605722, lon=-85.6800278, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.9893408, lon=-88.1012428, alt=801, tz=-6, dst='A')]

[Row(faa='04G', name='Lansdowne Airport', lat=41.1304722, lon=-80.6195833, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605722, lon=-85.6800278, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.9893408, lon=-88.1012428, alt=801, tz=-6, dst='A'),
 Row(faa='06N', name='Randall Airport', lat=41.431912, lon=-74.3915611, alt=523, tz=-5, dst='A'),
 Row(faa='09J', name='Jekyll Island Airport', lat=31.0744722, lon=-81.4277778, alt=11, tz=-4, dst='A'),
 Row(faa='0A9', name='Elizabethton Municipal Airport', lat=36.3712222, lon=-82.1734167, alt=1593, tz=-4, dst='A'),
 Row(faa='0G6', name='Williams County Airport', lat=41.4673056, lon=-84.5067778, alt=730, tz=-5, dst='A'),
 Row(faa='0G7', name='Finger Lakes Regional Airport', lat=42.8835647, lon=-76.7812318, alt=492, tz=-5, dst='A'),
 Row(faa='0P2', name='Shoestring Aviation Airfield', lat=39.7948244, lon=-76.6471914, alt=1000, tz=-5, dst='U'),
 Row(faa='0S9', name='Jefferson 

In [17]:
# 2.1.2
type(airports_df.head(4))               # List
# 2.1.3
type(airports_df.head(4)[0])            # Row


list

pyspark.sql.types.Row

In [18]:
# 2.2 Better display in columnar format

airports_df.show(5)                     # Tabular format
airports_df.show(5, False)              # Tabular format with extended columns


+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 5 rows

+---+-----------------------------+----------+-----------+----+---+---+
|faa|name                         |lat       |lon        |alt |tz |dst|
+---+-----------------------------+----------+-----------+----+---+---+
|04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |
|06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |
|06C|Schaumburg Regional          |41.9893408|-8

In [19]:
# 3.0 Have a look at the dataframe schema,
#     i.e. the structure of the DataFrame

airports_df.printSchema()
airports_df.dtypes

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



[('faa', 'string'),
 ('name', 'string'),
 ('lat', 'double'),
 ('lon', 'double'),
 ('alt', 'int'),
 ('tz', 'int'),
 ('dst', 'string')]

In [20]:
# 3.1 Column names
airports_df.columns

['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst']

In [21]:
# 3.2 How many rows:
airports_df.count()          # 1397

# 3.3 How many columns?
len(airports_df.columns)     # 7


1397

7

In [22]:
# 3.4 Full data summary
airports_df.describe().show()
# 3.4.1 Describe a particular column
airports_df.describe('dst').show(5)

+-------+------------------+--------------------+------------------+------------------+------------------+------------------+----+
|summary|               faa|                name|               lat|               lon|               alt|                tz| dst|
+-------+------------------+--------------------+------------------+------------------+------------------+------------------+----+
|  count|              1397|                1397|              1397|              1397|              1397|              1397|1397|
|   mean|             141.0|                null| 41.75029635989892|-103.6891285724532|1005.9169649248389|-6.420901932712956|null|
| stddev|197.46392075515973|                null|10.549872185047212|30.125313702028542|1521.2701426664623|1.8244027290430729|null|
|    min|               04G|Aberdeen Regional...|         19.721375|          -176.646|               -54|               -11|   A|
|    max|               ZYP|Zamperini Field A...|         72.270833|         174.11

In [23]:
###### C. Using Verbs
#         Select, filter, groupby, distinct, count()
#         It is more like 'dplyr' syntax, only that instead
#         of %>%, we have dot (.) .


# 4 Selecting Single/Multiple Columns
#   This does now work:   airports_df['faa'].show(5)
#    Following also does not work. Limitations
#    come because data is divided into chunks in various
#    servers.

airports_df[:2, :].show() 

TypeError: Invalid argument, not a string or column: slice(None, 2, None) of type <class 'slice'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [24]:
# 4.1
airports_df.select('faa').show(3)

#4.2 Both the following work:
airports_df.select('faa','dst').show(3)
airports_df.select(['faa','dst']).show(3)        # List of columns

+---+
|faa|
+---+
|04G|
|06A|
|06C|
+---+
only showing top 3 rows

+---+---+
|faa|dst|
+---+---+
|04G|  A|
|06A|  A|
|06C|  A|
+---+---+
only showing top 3 rows

+---+---+
|faa|dst|
+---+---+
|04G|  A|
|06A|  A|
|06C|  A|
+---+---+
only showing top 3 rows



In [25]:
# 4.3 Selecting Distinct values in Multiple Columns

airports_df.select('dst','tz').show()                 # Many repeating records
airports_df.select('dst','tz').distinct().show()      # Unique records

airports_df.select('dst','tz').count()                # 1397
airports_df.select('dst','tz').distinct().count()     # 22


+---+---+
|dst| tz|
+---+---+
|  A| -5|
|  A| -5|
|  A| -6|
|  A| -5|
|  A| -4|
|  A| -4|
|  A| -5|
|  A| -5|
|  U| -5|
|  A| -8|
|  A| -5|
|  U| -6|
|  A| -5|
|  U| -4|
|  A| -4|
|  A| -5|
|  A| -8|
|  U| -6|
|  A| -4|
|  U| -5|
+---+---+
only showing top 20 rows

+---+---+
|dst| tz|
+---+---+
|  A|  7|
|  A| -9|
|  A| -4|
|  U| -9|
|  A|  8|
|  A| -7|
|  N| -7|
|  N|-10|
|  A| -8|
|  U| -4|
|  A|  5|
|  A|-10|
|  A| -5|
|  U| -5|
|  U| -8|
|  N| -5|
|  U|-10|
|  A|  6|
|  A|-11|
|  A| -6|
+---+---+
only showing top 20 rows



1397

22

In [26]:
# 4.4 Like operator in where clause.
#        'where'  is an alias for 'filter' (see below)

airports_df.select('name'). \
            where("name like '%La%'").\
            show(3,False)

+-----------------------------+
|name                         |
+-----------------------------+
|Lansdowne Airport            |
|Finger Lakes Regional Airport|
|Frazier Lake Airpark         |
+-----------------------------+
only showing top 3 rows



In [27]:
# 5. Filtering data
# http://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.filter
#  We use the filter command to filter our DataFrame based
#  on the condition that tz must be equal to -5 and then
#  we are calculating how many records/rows are there in
#  the filtered output.
#  Syntax: filter(condition) 

# 5.1
airports_df.filter(airports_df.tz == -5) .show(3)
# 5.2
airports_df.filter(airports_df["tz"] == -5) .show(3)
# 5.3
airports_df.filter("tz == -5") .show(3)                          # Different syntax
# 5.4
from pyspark.sql.functions import col
airports_df.filter(col("tz") == -5) .show(3)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows

+---+--------------------+----------+-----------+----+---+---+
|faa|

In [28]:
# 5.5 Filter with regular expression:
airports_df.select('name'). \
            where(" name rlike  'pal$' " ). \
            show(3,truncate = False) 
# 5.6  where is an alias for filter  
airports_df.select('name'). \
            filter(" name rlike  'pal$' " ). \
            show(3,truncate = False) 
# 5.7    
airports_df.filter(" name rlike  'pal$' " ). \
            show(3,truncate = False) 

+-------------------+
|name               |
+-------------------+
|Mansfield Municipal|
|Wadsworth Municipal|
|Barnes Municipal   |
+-------------------+
only showing top 3 rows

+-------------------+
|name               |
+-------------------+
|Mansfield Municipal|
|Wadsworth Municipal|
|Barnes Municipal   |
+-------------------+
only showing top 3 rows

+---+-------------------+----------+-----------+---+---+---+
|faa|name               |lat       |lon        |alt|tz |dst|
+---+-------------------+----------+-----------+---+---+---+
|1B9|Mansfield Municipal|42.0001331|-71.1967714|122|-5 |A  |
|3G3|Wadsworth Municipal|41.0031572|-81.7564401|974|-5 |A  |
|BAF|Barnes Municipal   |42.1579472|-72.715875 |270|-4 |A  |
+---+-------------------+----------+-----------+---+---+---+
only showing top 3 rows



In [29]:
# 6. Combining verbs: select, filter and distinct
airports_df.select('dst', 'tz'). \
            filter(airports_df.tz == -5). \
            show(3)

airports_df.select('dst', 'tz'). \
            filter(airports_df.tz == -5). \
            distinct(). \
            show(3)

+---+---+
|dst| tz|
+---+---+
|  A| -5|
|  A| -5|
|  A| -5|
+---+---+
only showing top 3 rows

+---+---+
|dst| tz|
+---+---+
|  A| -5|
|  U| -5|
|  N| -5|
+---+---+



In [33]:
# 6.1 We can filter our data based on multiple conditions (AND or OR)
#     Logical Operators: & ==and,   | == or   ~ == not

airports_df.filter(                         \
                   (airports_df.tz == -5) & \
                   (airports_df.dst !="A")). \
                   show(3)   


+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|0P2|Shoestring Aviati...|39.7948244|-76.6471914|1000| -5|  U|
|1OH|     Fortman Airport|40.5553253|-84.3866186| 885| -5|  U|
|4I7|Putnam County Air...|39.6335556|-86.8138056| 842| -5|  U|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



In [39]:
# 7. groupby. Can apply sum, min, max, count

airports_df.groupby('tz'). \
           count(). \
           show(3)

airports_df.groupby('tz'). \
            agg({'lat' : 'count'}). \
            show(3)

+---+-----+
| tz|count|
+---+-----+
|-10|   26|
|-11|    2|
|  6|    1|
+---+-----+
only showing top 3 rows

+---+----------+
| tz|count(lat)|
+---+----------+
|-10|        26|
|-11|         2|
|  6|         1|
+---+----------+
only showing top 3 rows



In [ ]:
"""
Unpacking operator in python (*) : 
Ref: https://codeyarns.com/2012/04/26/unpack-operator-in-python/ 

def fox(a,b):  
    return (a *b)  

m = [3,4]  
fox(m)    
fox(*m)   

"""

In [42]:
# 7.1 One can take the average of columns by passing
#       an unpacked list of column names.

grObject = airports_df.groupby('tz')

avg_cols = ['lat', 'lon']
grObject.avg(*avg_cols).show(3)

+---+-----------------+-------------------+
| tz|         avg(lat)|           avg(lon)|
+---+-----------------+-------------------+
|-10|32.38998811538461|-146.58902384615382|
|-11|       56.8728385|       -169.9410275|
|  6|          29.5569|         -95.137497|
+---+-----------------+-------------------+
only showing top 3 rows



In [43]:
# 7.2 To call multiple aggregation functions at once, pass a dictionary.
#         The 'key' of dictionary becomes argument to 'value'.
#                             count(*)        avg(lat)      sum(lon)

grObject.agg({'*': 'count', 'lat': 'avg', 'lon':'sum'}).show(2)

+---+--------+-------------------+-----------------+
| tz|count(1)|           sum(lon)|         avg(lat)|
+---+--------+-------------------+-----------------+
|-10|      26|-3811.3146199999996|32.38998811538461|
|-11|       2|        -339.882055|       56.8728385|
+---+--------+-------------------+-----------------+
only showing top 2 rows



In [44]:
# 8. Create new columns in Spark using .withColumn() --mutate
#      New column: altInThousands . 
#      Product of two columns:  'alt' and  'lon' 

airports_df.withColumn('altInThousands', 
                       airports_df.alt*airports_df.lon
                      ).show(3)


+---+--------------------+----------+-----------+----+---+---+--------------+
|faa|                name|       lat|        lon| alt| tz|dst|altInThousands|
+---+--------------------+----------+-----------+----+---+---+--------------+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|-84166.8449652|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|-22619.5273392|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|-70569.0954828|
+---+--------------------+----------+-----------+----+---+---+--------------+
only showing top 3 rows



In [56]:
# 9. Save the new file with additional column in parquet form

xyz = airports_df.withColumn('altInThousands', airports_df.alt*airports_df.lon)
xyz.write.parquet("hdfs://localhost:9000/user/ashok/data_files/airports_extra.parquet")

In [57]:
# 9.1 Delete xyz from spark
import gc
del xyz
gc.collect()    # Delete all cache also

746

In [59]:
# 9.2 Read the stored parquet file
df = spark.read.parquet("hdfs://localhost:9000/user/ashok/data_files/airports_extra.parquet")
df.show(3)

+---+--------------------+----------+-----------+----+---+---+--------------+
|faa|                name|       lat|        lon| alt| tz|dst|altInThousands|
+---+--------------------+----------+-----------+----+---+---+--------------+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|-84166.8449652|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|-22619.5273392|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|-70569.0954828|
+---+--------------------+----------+-----------+----+---+---+--------------+
only showing top 3 rows



In [45]:
# 9.3 Read 'weather.csv file into spark from hadoop

URL_of_file= "hdfs://localhost:9000/user/ashok/data_files/nycflights/"
weather_df = spark.read.csv(path = URL_of_file + "weather.csv",
                            inferSchema = True,
                            header = True
                           )
weather_df.show(3)

+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
|origin|year|month|day|hour| temp| dewp|humid|wind_dir|wind_speed|    wind_gust|precip|pressure|visib|          time_hour|
+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
|   EWR|2013|    1|  1|   0|37.04|21.92|53.97|     230|  10.35702|11.9186514756|   0.0|  1013.9| 10.0|2013-01-01 05:30:00|
|   EWR|2013|    1|  1|   1|37.04|21.92|53.97|     230|  13.80936|15.8915353008|   0.0|    1013| 10.0|2013-01-01 06:30:00|
|   EWR|2013|    1|  1|   2|37.94|21.92|52.09|     230|  12.65858|14.5672406924|   0.0|  1012.6| 10.0|2013-01-01 07:30:00|
+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
only showing top 3 rows



In [46]:
# 10. Joins
# Refer: http://www.learnbymarketing.com/1100/pyspark-joins-by-example/
# For example, I can join the two titanic dataframes by the column PassengerId

# 10.1
airports_df.join(weather_df, airports_df.faa==weather_df.origin).show(3)
# 10.2
airports_df.join(weather_df, airports_df.faa==weather_df.origin, how = 'inner').show(3)
# 10.3
airports_df.join(weather_df, airports_df.faa==weather_df.origin, how = 'left').show(3)   # Could also use 'left_outer', 'right', 'full'


+---+-------------------+-------+----------+---+---+---+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
|faa|               name|    lat|       lon|alt| tz|dst|origin|year|month|day|hour| temp| dewp|humid|wind_dir|wind_speed|    wind_gust|precip|pressure|visib|          time_hour|
+---+-------------------+-------+----------+---+---+---+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
|EWR|Newark Liberty Intl|40.6925|-74.168667| 18| -5|  A|   EWR|2013|    1|  1|   0|37.04|21.92|53.97|     230|  10.35702|11.9186514756|   0.0|  1013.9| 10.0|2013-01-01 05:30:00|
|EWR|Newark Liberty Intl|40.6925|-74.168667| 18| -5|  A|   EWR|2013|    1|  1|   1|37.04|21.92|53.97|     230|  13.80936|15.8915353008|   0.0|    1013| 10.0|2013-01-01 06:30:00|
|EWR|Newark Liberty Intl|40.6925|-74.168667| 18| -5|  A|   EWR|2013|    1|  1|   2|37.94|21.92|52.09|     230|

### SQL queries against DataFrame

In [47]:
# 11. Many of the operations can be accessed by writing SQL queries in spark.sql().
# To make an existing Spark dataframe usable for spark.sql(), one needs to
#   register said dataframe as a temporary table.

# 11.1 As an example, we can register the two dataframes as temp tables then
#      join them through spark.sql().

airports_df.createOrReplaceTempView('dfa_temp')
weather_df.createOrReplaceTempView('dfw_temp')

In [48]:
# 11.2 Simple SQL query. SQLContext is no longer needed. 'spark'
#            session object can be used.

dfj = spark.sql('select * from dfa_temp' )
dfj.show(3)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



In [49]:
# 11.3 Now the SQL join

dfj = spark.sql('select * from dfa_temp a, dfw_temp b where a.faa = b.origin' )
dfj.show(3)


+---+-------------------+-------+----------+---+---+---+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
|faa|               name|    lat|       lon|alt| tz|dst|origin|year|month|day|hour| temp| dewp|humid|wind_dir|wind_speed|    wind_gust|precip|pressure|visib|          time_hour|
+---+-------------------+-------+----------+---+---+---+------+----+-----+---+----+-----+-----+-----+--------+----------+-------------+------+--------+-----+-------------------+
|EWR|Newark Liberty Intl|40.6925|-74.168667| 18| -5|  A|   EWR|2013|    1|  1|   0|37.04|21.92|53.97|     230|  10.35702|11.9186514756|   0.0|  1013.9| 10.0|2013-01-01 05:30:00|
|EWR|Newark Liberty Intl|40.6925|-74.168667| 18| -5|  A|   EWR|2013|    1|  1|   1|37.04|21.92|53.97|     230|  13.80936|15.8915353008|   0.0|    1013| 10.0|2013-01-01 06:30:00|
|EWR|Newark Liberty Intl|40.6925|-74.168667| 18| -5|  A|   EWR|2013|    1|  1|   2|37.94|21.92|52.09|     230|

In [50]:
# 12. Drop a columns

airports_df.drop('name').show(3)

# 12.1  Or drop multiple columns

columns_to_drop = ['name', 'lat']
xx =airports_df.drop(*columns_to_drop)
xx.show(3)

+---+----------+-----------+----+---+---+
|faa|       lat|        lon| alt| tz|dst|
+---+----------+-----------+----+---+---+
|04G|41.1304722|-80.6195833|1044| -5|  A|
|06A|32.4605722|-85.6800278| 264| -5|  A|
|06C|41.9893408|-88.1012428| 801| -6|  A|
+---+----------+-----------+----+---+---+
only showing top 3 rows

+---+-----------+----+---+---+
|faa|        lon| alt| tz|dst|
+---+-----------+----+---+---+
|04G|-80.6195833|1044| -5|  A|
|06A|-85.6800278| 264| -5|  A|
|06C|-88.1012428| 801| -6|  A|
+---+-----------+----+---+---+
only showing top 3 rows



In [ ]:
########### I am done ####################